# Evaluating models on the dataset

- Caricare i video
- Preprocessing dei video (resize e normalizzazione)
- Split train test
- Estrazione features con CNN
- passaggio features a LSTM per classificazione

## Imports

In [2]:
import tensorflow as tf
import pandas as pd
from tensorflow.keras.preprocessing.image import ImageDataGenerator

### Paths

In [ ]:
classes = pd.read_excel(filepath = 'wp8/excel_sheets/labels/labels.xlsx', sheet_name='labels')['labels'].tolist()
frames = ''

dataset.head()

In [ ]:
dataset = pd.concat(pd.read_excel("excel_sheets/dataset/dataset.xlsx", sheet_name=None, skiprows=1 ), ignore_index=True)
dataset.reset_index(drop=True)

### Hyperparameters

In [ ]:
TARGET_SIZE = (224,224)
BATCH_SIZE = 32

### Data preparation

In [5]:
def smart_resize_wrapper(x):
  return tf.keras.preprocessing.image.smart_resize(x, TARGET_SIZE)

datagen_args = dict(  rotation_range = 15,
                      width_shift_range = 0.2,
                      height_shift_range = 0.2,
                      zoom_range = 0.2,
                      horizontal_flip = True,
                      validation_split = 0.2,
                      preprocessing_function = smart_resize_wrapper
                        )
datagen = ImageDataGenerator(**datagen_args)
seed = 1

ds = datagen.flow_from_dataframe(
    dataset,
    directory=frames,
    x_col='frame',
    y_col='micro_labels',
    color_mode='rgb',
    classes=None,
    class_mode=None,
    batch_size=BATCH_SIZE,
    shuffle=True,
    seed=seed,
    validate_filenames=True,
)

## Feature extractor

In [ ]:
from tensorflow.keras.applications import EfficientNetV2L
input_shape = TARGET_SIZE + (3,)
model = EfficientNetV2L(include_top = False, weights = 'imagenet', input_shape = input_shape, pooling = 'avg')

features = model.predict(ds)